In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

In [3]:
text_feats = pd.read_csv('../../data/spooky-author/data/train_text_feats.csv')
# nb_score = pd.read_csv('../../data/spooky-author/data/train_nb_score.csv')
# nb_feats = pd.read_csv('../../data/spooky-author/data/test_nb_feats.csv')
# train_data = text_feats.merge(nb_feats, on='id', how='inner')
# train_data = train_data.merge(nb_score, on='id', how='inner')

dependent = pd.read_csv('../../data/spooky-author/download/train.csv', usecols=['id','author'])
mapper = {'EAP':0, 'HPL':1, 'MWS':2}
dependent['author'] = dependent['author'].map(lambda x : mapper[x])
train_data = dependent.merge(text_feats, on='id').drop('id', axis=1)
print('data shapes:', train_data.shape)
train_matrix = xgb.DMatrix(data = train_data.iloc[:,1:], label = train_data['author'])

data shapes: (19579, 23)


In [4]:
booster = {}
booster['booster'] = 'gbtree'
booster['nthread'] = 7
booster['max_depth'] = 4
booster['min_child_weight'] = 1
booster['subsample'] = 0.8
booster['colsample_bytree'] = 1.0
booster['colsample_bylevel'] = 0.9
booster['lambda'] = 1.0
booster['alpha'] = 0.0
booster['objective'] = 'multi:softprob'
booster['eval_metric'] = ['mlogloss']
booster['num_class'] = 3
booster['seed'] = 2017

In [6]:
params = {}
params['params'] = booster
params['dtrain'] = train_matrix
params['num_boost_round'] = 2000
params['folds'] =  KFold(n_splits=5, random_state=2017, shuffle=True).split(train_data)
params['early_stopping_rounds'] = 30
params['verbose_eval'] = 25
params['show_stdv'] = False
params['callbacks'] = [xgb.callback.reset_learning_rate([0.05] * 2000)]

In [7]:
model = xgb.cv(**params)

[0]	train-mlogloss:1.08922	test-mlogloss:1.08995
[25]	train-mlogloss:0.965599	test-mlogloss:0.980931
[50]	train-mlogloss:0.918601	test-mlogloss:0.946201
[75]	train-mlogloss:0.891676	test-mlogloss:0.930591
[100]	train-mlogloss:0.87294	test-mlogloss:0.922245
[125]	train-mlogloss:0.858304	test-mlogloss:0.917633
[150]	train-mlogloss:0.846193	test-mlogloss:0.914693
[175]	train-mlogloss:0.83569	test-mlogloss:0.912886
[200]	train-mlogloss:0.826153	test-mlogloss:0.91172
[225]	train-mlogloss:0.817457	test-mlogloss:0.911076
[250]	train-mlogloss:0.809135	test-mlogloss:0.910843
[275]	train-mlogloss:0.80125	test-mlogloss:0.910508
[300]	train-mlogloss:0.793645	test-mlogloss:0.910337
[325]	train-mlogloss:0.786236	test-mlogloss:0.910258
[350]	train-mlogloss:0.778953	test-mlogloss:0.910528
